In [1]:
from pandas import DataFrame
from DATA225utils import make_connection, dataframe_query

In [2]:
def make_table(table, sql):
    cursor_analytical.execute(f"DROP TABLE IF EXISTS {table}")
    cursor_analytical.execute(sql)

In [3]:
def display_table(table, order_by=''):
    sql = f"SELECT * FROM {table}"
    
    if order_by != '':
        sql = sql + " ORDER BY " + order_by
        
    _, df = dataframe_query(conn_analytical, sql)    
    return df

In [4]:
conn_operational = make_connection(config_file = 'nba.ini')
cursor_operational = conn_operational.cursor()

In [5]:
conn_analytical = make_connection(config_file = 'nba_analytical.ini')
cursor_analytical = conn_analytical.cursor()

In [6]:
sql = ( """
            CREATE TABLE calendar
            (
              date_key INT NOT NULL AUTO_INCREMENT,
              full_date DATE NOT NULL,
              day_of_week VARCHAR(10) NOT NULL,
              mnth INT NOT NULL,
              qtr NUMERIC NOT NULL,
              yr NUMERIC NOT NULL,
              PRIMARY KEY (date_key)
            )
        """
      )

make_table('calendar', sql)

DatabaseError: 3730 (HY000): Cannot drop table 'calendar' referenced by a foreign key constraint 'standings_ibfk_1' on table 'standings'.

In [ ]:
sql = ( """
            INSERT INTO nba_analytical.calendar(full_date, day_of_week , mnth, qtr, yr)
                SELECT DISTINCT(game_date), DAYNAME(game_date), 
                       MONTH(game_date), QUARTER(game_date), YEAR(game_date)
                FROM games;
        """
      )

cursor_operational.execute(sql)
conn_operational.commit()

display_table('calendar', 'full_date')

In [ ]:
sql = ( """
            CREATE TABLE team
            (
              team_key INT NOT NULL,
              team_name VARCHAR(25) NOT NULL,
              city VARCHAR(25) NOT NULL,
              state VARCHAR(25) NOT NULL,
              conference VARCHAR(5) NOT NULL,
              PRIMARY KEY (team_key)
            )
        """
      )

make_table('team', sql)

In [ ]:
sql = ( """
            INSERT INTO nba_analytical.team(team_key, team_name, city , state, conference)
                SELECT team_id, concat(city, " ", nickname), city, 
                       state, conference
                FROM teams;
        """
      )

cursor_operational.execute(sql)
conn_operational.commit()

display_table('team', 'team_name')

In [ ]:
sql = ( """
            CREATE TABLE player
            (
              player_key INT NOT NULL,
              player_name VARCHAR(25) NOT NULL,
              height NUMERIC,
              draft_year VARCHAR(20),
              PRIMARY KEY (player_key)
            )
        """
      )

make_table('player', sql)

In [ ]:
sql = ( """
            INSERT INTO nba_analytical.player(player_key, player_name, height , draft_year)
                SELECT player_id, player_name, height, draftyear
                FROM players;
        """
      )

cursor_operational.execute(sql)
conn_operational.commit()

display_table('player', 'player_name')

In [ ]:
sql = ( """
            CREATE TABLE game
            (
              game_key INT NOT NULL,
              date_key INT NOT NULL,
              home_team_key INT NOT NULL,
              away_team_key INT NOT NULL,
              home_score INT NOT NULL,
              away_score INT NOT NULL,
              venue VARCHAR(100) NOT NULL,
              PRIMARY KEY (game_key)
            )
        """
      )

make_table('game', sql)

In [ ]:
sql = ( """
            INSERT INTO nba_analytical.game(game_key, date_key, home_team_key , away_team_key,
                                            home_score, away_score, venue)
                SELECT G.game_id, C.date_key, home_team_id, visitor_team_id, 
                    home.pts AS home_pts, away.pts AS away_pts, arena AS venue 
                FROM games G
                INNER JOIN team_stats home
                ON home.game_id = G.game_id AND home.team_id = home_team_id
                INNER JOIN team_stats away
                ON away.game_id = G.game_id AND away.team_id = visitor_team_id
                INNER JOIN teams T
                ON T.team_id = home_team_id
                INNER JOIN nba_analytical.calendar C
                ON C.full_date =  G.game_date;
        """
      )

cursor_operational.execute(sql)
conn_operational.commit()

display_table('game', 'game_key')

In [ ]:
sql = ( """
            CREATE TABLE season
            (
              season_key INT NOT NULL AUTO_INCREMENT,
              season INT NOT NULL,
              PRIMARY KEY (season_key)
            )
        """
      )

make_table('season', sql)

In [ ]:
sql = ( """
            INSERT INTO nba_analytical.season(season)
                SELECT DISTINCT(season) FROM games;
        """
      )

cursor_operational.execute(sql)
conn_operational.commit()

display_table('season', 'season')

In [ ]:
sql = ( """
            CREATE TABLE player_stats
            (
              points INT,
              rebounds INT,
              assists NUMERIC,
              steals INT,
              blocks INT,
              turnovers INT,
              minutes_played FLOAT,
              fgm INT,
              fga INT,
              fg_pct FLOAT,
              ftm INT,
              fta NUMERIC,
              ft_pct FLOAT,
              fg3m NUMERIC,
              fg3a INT,
              fg3_pct FLOAT,
              player_key INT NOT NULL,
              game_key INT NOT NULL,
              team_key INT NOT NULL,
              PRIMARY KEY (player_key, game_key, team_key),
              FOREIGN KEY (player_key) REFERENCES player(player_key),
              FOREIGN KEY (game_key) REFERENCES game(game_key),
              FOREIGN KEY (team_key) REFERENCES team(team_key)
            )
        """
      )

make_table('player_stats', sql)

In [ ]:
sql = ( """
            INSERT INTO nba_analytical.player_stats(points, rebounds, assists, steals, blocks, 
                                                    turnovers, minutes_played, fgm, fga, fg_pct, 
                                                    ftm, fta, ft_pct, fg3m, fg3a, fg3_pct, 
                                                    player_key, game_key, team_key)
                SELECT pts, reb, ast, stl, blk, turnovers, seconds/60 as mintues_played, fgm, fga, fg_pct,
                    ftm, fta, ft_pct, fg3m, fg3a, fg3_pct, player_id, game_id, team_id 
                FROM nba.temp_player_game_stats;
        """
      )

cursor_operational.execute(sql)
conn_operational.commit()

display_table('player_stats', 'player_key, game_key, team_key')

In [ ]:
sql = ( """
            CREATE TABLE team_stats
            (
              points INT NOT NULL,
              rebounds INT NOT NULL,
              assists NUMERIC NOT NULL,
              fg_pct FLOAT NOT NULL,
              ft_pct FLOAT NOT NULL,
              fg3_pct FLOAT NOT NULL,
              game_key INT NOT NULL,
              team_key INT NOT NULL,
              PRIMARY KEY (game_key, team_key),
              FOREIGN KEY (game_key) REFERENCES Game(game_key),
              FOREIGN KEY (team_key) REFERENCES Team(team_key)
            )
        """
      )

make_table('team_stats', sql)

In [ ]:
sql = ( """
            INSERT INTO nba_analytical.team_stats(points, rebounds, assists, fg_pct, 
                                                    ft_pct, fg3_pct, game_key, team_key)
                SELECT pts, reb, ast, fg_pct, ft_pct, fg3_pct, game_id, team_id 
                FROM team_stats;
        """
      )

cursor_operational.execute(sql)
conn_operational.commit()

display_table('team_stats', 'game_key, team_key')

In [ ]:
sql = ( """
            CREATE TABLE standings
            (
              wins INT NOT NULL,
              losses INT NOT NULL,
              win_pct INT NOT NULL,
              date_key INT NOT NULL,
              team_key INT NOT NULL,
              PRIMARY KEY (date_key, team_key),
              FOREIGN KEY (date_key) REFERENCES Calendar(date_key),
              FOREIGN KEY (team_key) REFERENCES Team(team_key)
            )
        """
      )

make_table('standings', sql)

In [ ]:
sql = ( """
            INSERT INTO nba_analytical.standings(wins, losses, win_pct, date_key, 
                                                    team_key)
                SELECT games_won, games_lost, win_pct, date_key, team_id
                FROM nba.ranking
                INNER JOIN nba_analytical.calendar
                ON full_date = standings_date;
        """
      )

cursor_operational.execute(sql)
conn_operational.commit()

display_table('standings', 'date_key, team_key')

In [ ]:
cursor_operational.close()
cursor_analytical.close()
conn_operational.close()
conn_analytical.close()